In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve

In [17]:
df = w.wrangle_coll_stage1(year_to_retrieve, app_token)
df.head()

CSV file for 2022 already exists. Loading data from the CSV.


,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,on_street_name,borough,zip_code,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,injuries_count,deaths_count,injuries,deaths
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.851690,-73.952380,CROSS BRONX EXPY,NaN,7024,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified,0,0,False,False
1,2022-11-17 10:50:00,2022-11-17,10:50,4582618,40.802204,-73.967804,BROADWAY,Manhattan,10025,Station Wagon/Sport Utility Vehicle,Passing or Lane Usage Improper,Box Truck,Passing or Lane Usage Improper,0,0,False,False
2,2022-11-17 14:40:00,2022-11-17,14:40,4582613,40.756664,-73.968860,East 52nd Street,MANHATTAN,10022,Taxi,Passing Too Closely,Flat Bed,Unspecified,0,0,False,False
3,2022-11-17 15:00:00,2022-11-17,15:00,4583338,40.865364,-73.901344,Grand Avenue,BRONX,10468,Sedan,Driver Inattention/Distraction,Bus,Unspecified,0,0,False,False
4,2022-11-17 14:36:00,2022-11-17,14:36,4583745,40.714336,-73.946370,Metropolitan Avenue,BROOKLYN,11211,Box Truck,Following Too Closely,Sedan,Unspecified,0,0,False,False


In [4]:
df = w.get_data(year_to_retrieve, app_token, max_req)
df = w.date_time(df)
df = w.initial_reorder_cols(df)
df = w.drop_3plus(df)
df = w.single_coll(df)
df = w.motorist_only(df)
df = w.second_drop(df)
df = w.null_zero_loc(df)

CSV file for 2022 already exists. Loading data from the CSV.


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54214 entries, 0 to 58536
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 54214 non-null  datetime64[ns]
 1   crash_date                     54214 non-null  datetime64[ns]
 2   crash_time                     54214 non-null  object        
 3   collision_id                   54214 non-null  int64         
 4   latitude                       48263 non-null  float64       
 5   longitude                      48263 non-null  float64       
 6   location                       48263 non-null  object        
 7   on_street_name                 41609 non-null  object        
 8   borough                        34461 non-null  object        
 9   zip_code                       34453 non-null  float64       
 10  number_of_persons_injured      54214 non-null  int64         
 11  number_of_persons_ki

In [58]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,45821,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,54204,int64
4,latitude,5951,11.0,0,0,23107,float64
5,longitude,5951,11.0,0,0,19572,float64
6,location,5951,11.0,0,0,27538,object
7,on_street_name,12605,23.0,0,0,3509,object
8,borough,19753,36.0,0,0,5,object
9,zip_code,19761,36.0,0,0,200,float64


In [8]:
missing_df = df[(df['latitude'].notnull()) & (df[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]


In [10]:
len(missing_df)

27095

___
We have identified any row with valid lat and long and missing 'zip_code', 'borough', 'on_street_name'.
___

We can now run make api calls to google geocoding and get the missing information using the lat and long. 

In [32]:
import pandas as pd
import requests

def fill_missing_geocode_info(df, api_key, save_path, save_interval=1000):
    # Create a list to store geocode data
    geocode_data = []
    
    # Initialize a counter to keep track of API calls
    api_call_count = 0
    
    # Iterate through rows of the original DataFrame
    for index, row in df.iterrows():
        # Check if ZIP code, borough, and on-street name are missing
        if pd.isna(row['zip_code']) or pd.isna(row['borough']) or pd.isna(row['on_street_name']):
            lat = row['latitude']
            lng = row['longitude']
            
            # Make the API request
            url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                
                # Initialize variables to store the address components
                zipcode = None
                borough = None
                street_address = None
                
                # Parse the response to extract address components
                for result in data.get('results', []):
                    for component in result.get('address_components', []):
                        if 'postal_code' in component['types']:
                            zipcode = component['short_name']
                        if 'sublocality_level_1' in component['types']:
                            borough = component['long_name']
                        if 'route' in component['types']:
                            street_address = component['long_name']
                
                # Fill in missing information in the DataFrame
                if pd.isna(row['zip_code']):
                    df.at[index, 'zip_code'] = zipcode
                if pd.isna(row['borough']):
                    df.at[index, 'borough'] = borough
                if pd.isna(row['on_street_name']):
                    df.at[index, 'on_street_name'] = street_address
                
                # Append geocode data to the list
                geocode_data.append({'latitude': lat, 'longitude': lng, 'geocode_json': data})
                
                # Increment the API call counter
                api_call_count += 1
                
                # Check if it's time to save progress
                if api_call_count % save_interval == 0:
                    # Save the DataFrame and geocode data to CSV files
                    df.to_csv(save_path, index=False)
                    geocode_df = pd.DataFrame(geocode_data)
                    geocode_df.to_csv(f'geocode_data_{api_call_count}.csv', index=False)
                    print(f"Progress saved after {api_call_count} API calls.")
            else:
                print(f"Error: {response.status_code} for row {index}")
    
    # Save the final DataFrame and geocode data to CSV files
    df.to_csv(save_path, index=False)
    geocode_df = pd.DataFrame(geocode_data)
    geocode_df.to_csv(f'geocode_data_{api_call_count}.csv', index=False)
    
    return df

Make the api calls and update the columns

In [33]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geo_df_1.csv'

In [ ]:
# Fill missing geocode information in the DataFrame
first_run = fill_missing_geocode_info(missing_df, api_key, save_path, save_interval=1000)


Optional: if the fill_missing_geocode_info function stalls or otherwise fails to load the dataframe to filled_df, then we can load the csv files created by fill_missing_geocode_info as a safety mechanism (save_interval). 

In [25]:
first_run = pd.read_csv('geo_df_1.csv')

In [26]:
len(first_run)

27095

Lets see the new info


In [92]:
# Example usage:
info_dataframe = exp.create_info_dataframe(first_run)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,1341,5.0,0,0,4312,object
8,borough,1783,7.0,0,0,10,object
9,zip_code,1783,7.0,0,0,205,float64


for some reason we have a difference in nulls on street name, borough and zip_code despite running the api calls.

we should have a maximum of 5951 missing street name, borough and zip_code. Yet we have 5972 missing zip codes.  

update: I found out that the fill_missing_geocode_info function had stalled out and did not get all of the missing geocode info. The discrepancy in null values form lat and long and the street name, borough and zip_code is caused by this. I will have to create a dataframe containing rows with lat and long that are missing any street name, borough and zip_code. Run that df through the fill_missing_geocode_info and merge the first run with the second run. 

the street name has 2081 which is lower than I expected as I seem to remember checking and finding out that all rows with missing lat and long also were missing street name. I will have to double check this later.

In [93]:
missing_first_run = first_run[(first_run['latitude'].notnull()) & (first_run[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]

In [94]:
len(missing_first_run)

3081

we ill have to run missing_first_run through fill_missing_geocode_info to get the missing geo location info. 

In [34]:
second_run = fill_missing_geocode_info(missing_first_run, api_key, save_path, save_interval=1000)

Progress saved after 1000 API calls.
Progress saved after 2000 API calls.
Progress saved after 3000 API calls.


In [27]:
second_run = pd.read_csv('geo_df_2.csv')

Lets confirm that we filled the missing info

In [98]:
second_run.head(1)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,NaN,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


Lets now load the first run df with the second run df

In [28]:
len(first_run), len(second_run)

(27095, 3081)

lets print the null count of the first run so we can compare it later

now we must concat the 2 dataframes but first we need to remove any rows that exist in the filled_df (second api run) from the first_run dataframe. 

In [29]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = second_run['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
first_run = first_run[~first_run['collision_id'].isin(collision_ids_to_remove)]

# Concatenate the two dataframes
combined_one_tow = pd.concat([second_run, first_run], ignore_index=True)

In [30]:
len(first_run)

24014

In [31]:
len(combined_one_tow)

27095

In [114]:
# Example usage:
filled_df_info = exp.create_info_dataframe(combined_one_tow)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,0,0.0,0,0,4430,object
8,borough,1,0.0,0,0,10,object
9,zip_code,0,0.0,0,0,205,float64


In [32]:
missing_df_second = combined_one_tow[(combined_one_tow['latitude'].notnull()) & (combined_one_tow[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]

In [33]:
len(missing_df_second)

1

In [126]:
missing_df_second

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,NaN,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [109]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geo_df_3.csv'

In [127]:
third_run = fill_missing_geocode_info(missing_df_second, api_key, save_path, save_interval=1000)

In [34]:
third_run = pd.read_csv('geo_df_3.csv')

In [135]:
third_run

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [59]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = third_run['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
combined_one_tow = combined_one_tow[~combined_one_tow['collision_id'].isin(collision_ids_to_remove)]

# Concatenate the two dataframes
combined_three = pd.concat([third_run, combined_one_tow], ignore_index=True)

In [60]:
len(combined_three)

27095

In [157]:
combined_one_tow.head(1)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
1,2022-11-17 10:50:00,2022-11-17,10:50,4582618,40.802204,-73.967804,"{'latitude': '40.802204', 'longitude': '-73.96...",BROADWAY,Manhattan,10025.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Passing or Lane Usage Improper,Box Truck,Passing or Lane Usage Improper


In [155]:
third_run.head(1)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [156]:
combined_three.head(1)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [79]:
combined_three.to_csv('final_geo_filled_errors.csv')

___
Load the final geo
___

In [5]:
combined_three = pd.read_csv('final_geo_filled_errors.csv', index_col=0)

In [6]:
# Remove duplicates in 'df' when all columns have the same information
df.drop_duplicates(keep='first', inplace=True)
combined_three.drop_duplicates(keep='first', inplace=True)

In [7]:
# Create 'df_b' containing rows with matching 'collision_id' in 'combined_three'
df_b = df[df['collision_id'].isin(combined_three['collision_id'])]

# Create 'df_a' containing rows not matched in 'combined_three'
df_a = df[~df['collision_id'].isin(combined_three['collision_id'])]


In [8]:
len(df_a), len(df_b), len(combined_three)

(27113, 27091, 27091)

In [67]:
# Example usage:
filled_df_info = exp.create_info_dataframe(combined_three)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,0,0.0,0,0,4430,object
8,borough,1,0.0,0,0,10,object
9,zip_code,0,0.0,0,0,205,float64


In [ ]:
combined_three[combined_three['collision_id'] == 4562046]

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [6]:
len(df), len(combined_three)

(54214, 27095)

In [44]:
combined_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27095 entries, 0 to 27094
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   crash_datetime                 27095 non-null  object 
 1   crash_date                     27095 non-null  object 
 2   crash_time                     27095 non-null  object 
 3   collision_id                   27095 non-null  int64  
 4   latitude                       27095 non-null  float64
 5   longitude                      27095 non-null  float64
 6   location                       27095 non-null  object 
 7   on_street_name                 27095 non-null  object 
 8   borough                        27094 non-null  object 
 9   zip_code                       27095 non-null  float64
 10  number_of_persons_injured      27095 non-null  int64  
 11  number_of_persons_killed       27095 non-null  int64  
 12  number_of_motorist_injured     27095 non-null 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54214 entries, 0 to 58536
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 54214 non-null  datetime64[ns]
 1   crash_date                     54214 non-null  datetime64[ns]
 2   crash_time                     54214 non-null  object        
 3   collision_id                   54214 non-null  int64         
 4   latitude                       48263 non-null  float64       
 5   longitude                      48263 non-null  float64       
 6   location                       48263 non-null  object        
 7   on_street_name                 41609 non-null  object        
 8   borough                        34461 non-null  object        
 9   zip_code                       34453 non-null  float64       
 10  number_of_persons_injured      54214 non-null  int64         
 11  number_of_persons_ki

In [9]:
# Merge the two DataFrames on 'collision_id' and select specific columns
merged_df = df_b.merge(combined_three[['collision_id', 'on_street_name', 'borough', 'zip_code']],
                     on='collision_id', how='left')


In [15]:
len(df_b), len(merged_df)

(27091, 27091)

In [12]:
df_c = df_b.copy()

In [13]:
len(df_b), len(df_c)

(27091, 27091)

In [10]:
len(merged_df)

54204

In [ ]:
merged_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name_x,borough_x,zip_code_x,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name_y,borough_y,zip_code_y
0,2022-01-01 04:30:00,2022-01-01,04:30:00,4491626,NaN,NaN,NaN,Southern parkway,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified,NaN,NaN,NaN
1,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing,GRAND CENTRAL PKWY,Queens,11368.0
2,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,HENRY HUDSON PARKWAY,Manhattan,10027.0
3,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified,Saint Ann's Avenue,BRONX,10454.0
4,2022-01-01 10:00:00,2022-01-01,10:00:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54217,2022-12-31 23:05:00,2022-12-31,23:05:00,4598127,40.737710,-73.974266,"{'latitude': '40.73771', 'longitude': '-73.974...",FDR DRIVE,NaN,NaN,1,0,1,0,Sedan,Other Vehicular,Van,Driver Inexperience,FDR DRIVE,Manhattan,10016.0
54218,2022-12-31 02:00:00,2022-12-31,02:00:00,4647650,40.648243,-73.929000,"{'latitude': '40.648243', 'longitude': '-73.92...",TILDEN AVENUE,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified,TILDEN AVENUE,Brooklyn,11203.0
54219,2022-12-31 12:20:00,2022-12-31,12:20:00,4599098,40.657093,-73.950260,"{'latitude': '40.657093', 'longitude': '-73.95...",NOSTRAND AVENUE,NaN,NaN,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unsafe Lane Changing,Station Wagon/Sport Utility Vehicle,Unspecified,NOSTRAND AVENUE,Brooklyn,11225.0
54220,2022-12-31 12:39:00,2022-12-31,12:39:00,4606651,40.667210,-73.928505,"{'latitude': '40.66721', 'longitude': '-73.928...",NaN,BROOKLYN,11213.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,Rochester Avenue,BROOKLYN,11213.0


In [65]:
df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,04:30:00,4491626,NaN,NaN,NaN,Southern parkway,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58531,2022-12-31 23:05:00,2022-12-31,23:05:00,4598127,40.737710,-73.974266,"{'latitude': '40.73771', 'longitude': '-73.974...",FDR DRIVE,NaN,NaN,1,0,1,0,Sedan,Other Vehicular,Van,Driver Inexperience
58532,2022-12-31 02:00:00,2022-12-31,02:00:00,4647650,40.648243,-73.929000,"{'latitude': '40.648243', 'longitude': '-73.92...",TILDEN AVENUE,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified
58534,2022-12-31 12:20:00,2022-12-31,12:20:00,4599098,40.657093,-73.950260,"{'latitude': '40.657093', 'longitude': '-73.95...",NOSTRAND AVENUE,NaN,NaN,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unsafe Lane Changing,Station Wagon/Sport Utility Vehicle,Unspecified
58535,2022-12-31 12:39:00,2022-12-31,12:39:00,4606651,40.667210,-73.928505,"{'latitude': '40.66721', 'longitude': '-73.928...",NaN,BROOKLYN,11213.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified


In [68]:
# Update the columns in 'df' with values from 'combined_three'
df_b['on_street_name'] = merged_df['on_street_name_y']
df_b['borough'] = merged_df['borough_y']
df_b['zip_code'] = merged_df['zip_code_y']

In [21]:
# Update 'on_street_name' column in 'df' if 'merged_df' value is not null
df_c['on_street_name'] = merged_df.apply(lambda row: row['on_street_name_y'] if pd.notnull(row['on_street_name_y']) else df_c['on_street_name'], axis=1)

# Update 'borough' column in 'df' if 'merged_df' value is not null
df_c['borough'] = merged_df.apply(lambda row: row['borough_y'] if pd.notnull(row['borough_y']) else df_c['borough'], axis=1)

# Update 'zip_code' column in 'df' if 'merged_df' value is not null
df_c['zip_code'] = merged_df.apply(lambda row: row['zip_code_y'] if pd.notnull(row['zip_code_y']) else df_c['zip_code'], axis=1)


In [19]:
# Find 'collision_id' values present in 'df' but not in 'combined_three'
missing_collision_ids_df = df_b[~df_b['collision_id'].isin(merged_df['collision_id'])]

# Find 'collision_id' values present in 'combined_three' but not in 'df'
missing_collision_ids_combined_three = merged_df[~merged_df['collision_id'].isin(df_b['collision_id'])]

# Display the number of missing 'collision_id' values in each DataFrame
print(f"Number of missing 'collision_id' values in 'df': {len(missing_collision_ids_df)}")
print(f"Number of missing 'collision_id' values in 'combined_three': {len(missing_collision_ids_combined_three)}")


Number of missing 'collision_id' values in 'df': 0
Number of missing 'collision_id' values in 'combined_three': 0


In [97]:
# Check for duplicates in 'df'
duplicates_df = df[df.duplicated(subset='collision_id', keep=False)]

# Check for duplicates in 'combined_three'
duplicates_combined_three = combined_three[combined_three.duplicated(subset='collision_id', keep=False)]

# Display the number of duplicates in each DataFrame
print(f"Number of duplicates in 'df': {len(duplicates_df)}")
print(f"Number of duplicates in 'combined_three': {len(duplicates_combined_three)}")


Number of duplicates in 'df': 0
Number of duplicates in 'combined_three': 8


In [98]:
duplicate_rows_df = df[df.duplicated(subset='collision_id', keep=False)]
duplicate_rows_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2


In [20]:
# Update 'on_street_name' column in 'df' if 'merged_df' value is not null
df_c['on_street_name'] = merged_df['on_street_name_y'].fillna(df_c['on_street_name'])

# Update 'borough' column in 'df' if 'merged_df' value is not null
df_b['borough'] = merged_df['borough_y'].fillna(df_c['borough'])

# Update 'zip_code' column in 'df' if 'merged_df' value is not null
df_c['zip_code'] = merged_df['zip_code_y'].fillna(df_c['zip_code'])


/var/folders/6c/7ht5b3716k32pz2lmh_pzp8m0000gp/T/ipykernel_19470/3725694749.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b['borough'] = merged_df['borough_y'].fillna(df_c['borough'])


In [75]:
df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,04:30:00,4491626,NaN,NaN,NaN,Southern parkway,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,Queens,11368.0,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,Manhattan,10027.0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",Saint Ann's Avenue,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58531,2022-12-31 23:05:00,2022-12-31,23:05:00,4598127,40.737710,-73.974266,"{'latitude': '40.73771', 'longitude': '-73.974...",NaN,NaN,NaN,1,0,1,0,Sedan,Other Vehicular,Van,Driver Inexperience
58532,2022-12-31 02:00:00,2022-12-31,02:00:00,4647650,40.648243,-73.929000,"{'latitude': '40.648243', 'longitude': '-73.92...",NaN,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified
58534,2022-12-31 12:20:00,2022-12-31,12:20:00,4599098,40.657093,-73.950260,"{'latitude': '40.657093', 'longitude': '-73.95...",NaN,NaN,NaN,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unsafe Lane Changing,Station Wagon/Sport Utility Vehicle,Unspecified
58535,2022-12-31 12:39:00,2022-12-31,12:39:00,4606651,40.667210,-73.928505,"{'latitude': '40.66721', 'longitude': '-73.928...",NaN,NaN,NaN,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified


In [53]:
df.to_csv('final_geo_filled.csv')

In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54204 entries, 0 to 54203
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 54204 non-null  datetime64[ns]
 1   crash_date                     54204 non-null  datetime64[ns]
 2   crash_time                     54204 non-null  object        
 3   collision_id                   54204 non-null  int64         
 4   latitude                       48254 non-null  float64       
 5   longitude                      48254 non-null  float64       
 6   location                       48254 non-null  object        
 7   on_street_name_x               41599 non-null  object        
 8   borough_x                      34456 non-null  object        
 9   zip_code_x                     34448 non-null  float64       
 10  number_of_persons_injured      54204 non-null  int64         
 11  number_of_perso

In [ ]:
len(df), len(combined_three)

(54214, 27095)

In [47]:
merged_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name_x,borough_x,zip_code_x,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name_y,borough_y,zip_code_y
0,2022-01-01 04:30:00,2022-01-01,04:30:00,4491626,NaN,NaN,NaN,Southern parkway,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified,NaN,NaN,NaN
1,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing,GRAND CENTRAL PKWY,Queens,11368.0
2,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,HENRY HUDSON PARKWAY,Manhattan,10027.0
3,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified,Saint Ann's Avenue,BRONX,10454.0
4,2022-01-01 10:00:00,2022-01-01,10:00:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54217,2022-12-31 23:05:00,2022-12-31,23:05:00,4598127,40.737710,-73.974266,"{'latitude': '40.73771', 'longitude': '-73.974...",FDR DRIVE,NaN,NaN,1,0,1,0,Sedan,Other Vehicular,Van,Driver Inexperience,FDR DRIVE,Manhattan,10016.0
54218,2022-12-31 02:00:00,2022-12-31,02:00:00,4647650,40.648243,-73.929000,"{'latitude': '40.648243', 'longitude': '-73.92...",TILDEN AVENUE,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified,TILDEN AVENUE,Brooklyn,11203.0
54219,2022-12-31 12:20:00,2022-12-31,12:20:00,4599098,40.657093,-73.950260,"{'latitude': '40.657093', 'longitude': '-73.95...",NOSTRAND AVENUE,NaN,NaN,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unsafe Lane Changing,Station Wagon/Sport Utility Vehicle,Unspecified,NOSTRAND AVENUE,Brooklyn,11225.0
54220,2022-12-31 12:39:00,2022-12-31,12:39:00,4606651,40.667210,-73.928505,"{'latitude': '40.66721', 'longitude': '-73.928...",NaN,BROOKLYN,11213.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,Rochester Avenue,BROOKLYN,11213.0


In [36]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = combined_three['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
df = df[~df['collision_id'].isin(collision_ids_to_remove)]


In [49]:
len(df)

54214

In [52]:
df.head()

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,04:30:00,4491626,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,Queens,11368.0,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,Manhattan,10027.0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,"{'latitude': '40.806107', 'longitude': '-73.91...",Saint Ann's Avenue,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00:00,4491791,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified


In [145]:
# Concatenate the two dataframes
df = pd.concat([combined_three, df], ignore_index=True)

In [146]:
len(df)

54214

In [150]:
df.to_csv('df_geo_cleaned.csv')

In [17]:
# Example usage:
filled_df_info = exp.create_info_dataframe(merged_df)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name_x,11865,44.0,0,0,1870,object
8,borough_x,15562,57.0,0,0,5,object
9,zip_code_x,15570,57.0,0,0,193,float64


In [ ]:
# Example usage:
filled_df_info = exp.create_info_dataframe(df_b)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,11865,44.0,0,0,1870,object
8,borough,14494,54.0,0,0,10,object
9,zip_code,15570,57.0,0,0,193,float64


In [19]:
# Example usage:
filled_df_info = exp.create_info_dataframe(df_c)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,14494,54.0,0,0,3182,object
8,borough,14494,54.0,0,0,10,object
9,zip_code,14494,54.0,0,0,197,float64


filled_df contains only the filtered df rows with missing location columns. 
we have to find a way to merge the filled df with the original df while keeping the updated values from filled df.

